In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 1
sim = 0
Min_Entries = 1000
N_files = 1
Do_save = 1

In [3]:
isigma = 2
N_layers = 4
N_hists = 4
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/"
file_names="my-m_ee_Run14AuAu_83rd_new_19842_1065runs" 
hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/deltas/" 
    file_path="../sim/output/Newembed/sngl/" 
    file_path="../sim/output/Newembed/deltas/250301/" 
    file_names="dalitz_50M_v3"#"bbbar_10M_v0" #piminus_pythia_1M_v0 piminus_DCA_50M_V3 photon_100M_V0 piminus_DCA_50M_V1 piminus_DCA_50M_V0_100mrad ccbar_50M_v0 piminus_DCA_50M_V0
    file_names="lepton_50M_v1"
    file_names="../../dca/piminus_50M_v0"

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ilayer in range(N_layers):
    hist_select_3D0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
    hists_read.append(hist_select_3D0)

infile.Close()

In [ ]:
delta_z = 5
z_bins = [i for i in range(9,30,delta_z)]
z_means = array('d', [i-19.5 for i in z_bins])
z_error = array('d', [0.5*delta_z for i in z_bins])
print(z_means, z_error)
pad_limits = [[2,3.7],[-1,0.7]]

array('d', [-10.5, -9.5, -8.5, -7.5, -6.5, -5.5, -4.5, -3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5]) array('d', [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])


In [7]:
def sdphi_fitter(ihits, sigma=isigma):

    iHist = hists_read[ihits][iOption0+is_sigmalised*2]

    Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg = [], [], [], [], [], [], []

    for iz in z_bins:

        phi_z =  array( 'd' )
        phi_z_err =  array( 'd' )
        Mean_z =  array( 'd' )
        Sigma_z =  array( 'd' )
        Mean_z_err =  array( 'd' )
        Sigma_z_err =  array( 'd' )
        int_fg_local =  array( 'd' )

        N_ybins_loc = iHist.GetNbinsY()
        for iy in range(N_ybins_loc):
            proj_local = iHist.ProjectionX(iHist.GetName()+f"_{iz}_{iy}", iy+1, iy+1, iz+1, iz+1)
            if proj_local.GetEntries() < Min_Entries:
                continue
            
            phi = iHist.GetYaxis().GetBinCenter(iy+1)
            phi_z.append(phi)
            phi_z_err.append(iHist.GetYaxis().GetBinWidth(iy+1)/2.)

            c1 = root.TCanvas("c1"+proj_local.GetName(),"c1"+proj_local.GetName(),720,360)
            gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", -0.0025, 0.0025)
            proj_local.Draw()
            proj_local.Fit(gaus, "Q", "", -0.0025, 0.0025)
            c1.Update()
            if Do_save: c1.SaveAs("output/dynamic_delta_fits/" + proj_local.GetName() + ".png")

            Mean_z.append(gaus.GetParameter(1))
            Sigma_z.append(gaus.GetParameter(2))
            Mean_z_err.append(min(gaus.GetParError(1)*1000, gaus.GetParameter(2)/3))
            Sigma_z_err.append(min(gaus.GetParError(2), 0.05*gaus.GetParameter(2)))


            int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-0.002,gaus.GetParameter(1)+0.002))
            #int_bg_local.append(2*gaus.Integral(gaus.GetParameter(1)+sigma*gaus.GetParameter(2),0.0025))

            #m2_pos.Draw("same") m2_neg.Draw("same")
            #c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )
        Mean_local.append(Mean_z)
        Mean_local_err.append(Mean_z_err)
        Sigma_local.append(Sigma_z)
        Sigma_local_err.append(Sigma_z_err)
        phi_local.append(phi_z)
        phi_local_err.append(phi_z_err)
        int_fg.append(int_fg_local)

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg]


In [8]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter, [i for i in range(Ntr)])
pool.close()

In [9]:
print(len(output_array[0][0]))

21


In [10]:
Means, Means_err, Sigmas, Sigmas_err, phis, phis_err, int_fg = [], [], [], [], [], [], []
for i in range(N_layers):
    Means.append(output_array[i][0])
    Means_err.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigmas_err.append(output_array[i][3])
    phis.append(output_array[i][4])
    phis_err.append(output_array[i][5])
    int_fg.append(output_array[i][6])
    
print("Means", len(Means), len(Means[0]), len(Means[0][1]))

Means 4 21 11


In [11]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_0iter = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(2):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_0iter_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.0)
                fit_functions1[-1].SetParLimits(1,0.0001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions_0iter.append(fit_functions0)

In [12]:
g_sigma = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = 0., 0.003
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_sigma_layer = []
    h2.append(Format_Pad_old(pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Sigmas[ilayer])):
        n_phi = len(Sigmas[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Sigmas[ilayer][ized],phis_err[ilayer][ized], Sigmas_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [13]:
fit_mean_functions = []
for ilayer in range(N_layers):
    fit_functions = []
    for ized in range(len(z_bins)):
        fit_functions.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer],"[0]/10000*sin(x)+[1]/10000*cos(x)+[2]",-1.5,4.5))
        fit_functions[-1].FixParameter(2,0.00)
        #fit_functions[-1].FixParameter(1,0.00)
    fit_mean_functions.append(fit_functions)

In [14]:
g_mean = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = -0.002, 0.002
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_mean_layer = []
    h2.append(Format_Pad_old( pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Means[ilayer])):
        n_phi = len(Means[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Means[ilayer][ized],phis_err[ilayer][ized], Means_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Fit(fit_mean_functions[ilayer][ized],"Q", "", pad_limits[ilayer%2][0]+0.3, pad_limits[ilayer%2][1])
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [15]:
for ilayer in range(N_layers):
    for ized in range(len(z_bins)):
        print(f"Layer {ilayer}, z_bin {ized}, dx: {fit_mean_functions[ilayer][ized].GetParameter(0):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(0):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, dy: {fit_mean_functions[ilayer][ized].GetParameter(1):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(1):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, c: {fit_mean_functions[ilayer][ized].GetParameter(2):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(2):.6f}")

Layer 0, z_bin 0, dx: 42.297130 +/- 13.683582
Layer 0, z_bin 0, dy: 19.893232 +/- 3.500130
Layer 0, z_bin 0, c: 0.000000 +/- 0.000000
Layer 0, z_bin 1, dx: 23.948303 +/- 3.630891
Layer 0, z_bin 1, dy: 13.718883 +/- 1.207479
Layer 0, z_bin 1, c: 0.000000 +/- 0.000000
Layer 0, z_bin 2, dx: 23.464870 +/- 3.762257
Layer 0, z_bin 2, dy: 12.281091 +/- 1.224314
Layer 0, z_bin 2, c: 0.000000 +/- 0.000000
Layer 0, z_bin 3, dx: 27.097954 +/- 3.227643
Layer 0, z_bin 3, dy: 12.298777 +/- 1.173840
Layer 0, z_bin 3, c: 0.000000 +/- 0.000000
Layer 0, z_bin 4, dx: 19.251991 +/- 3.131971
Layer 0, z_bin 4, dy: 8.188165 +/- 1.055087
Layer 0, z_bin 4, c: 0.000000 +/- 0.000000
Layer 0, z_bin 5, dx: 22.419729 +/- 2.907563
Layer 0, z_bin 5, dy: 7.981570 +/- 1.063223
Layer 0, z_bin 5, c: 0.000000 +/- 0.000000
Layer 0, z_bin 6, dx: 18.928937 +/- 2.507100
Layer 0, z_bin 6, dy: 4.606283 +/- 0.947143
Layer 0, z_bin 6, c: 0.000000 +/- 0.000000
Layer 0, z_bin 7, dx: 18.026526 +/- 2.269757
Layer 0, z_bin 7, dy: 3.31

In [16]:
# Example for plotting means in 3D for ilayer=0
ilayer = 0
g3d = root.TGraph2D()
point = 0
for ized, z in enumerate(z_bins):
    n_phi = len(Means[ilayer][ized])
    for iphi in range(n_phi):
        phi = phis[ilayer][ized][iphi]
        mean = Means[ilayer][ized][iphi]
        g3d.SetPoint(point, phi, z_means[ized], mean)
        point += 1
# Define the fit function: ([0]+[1]*zed)*sin(phi)+([2]+[3]*zed)*cos(phi)+[4]
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", -2, 4, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these

c3d = root.TCanvas("c3d", "3D Mean", 800, 600)


g3d.SetTitle("Mean vs #phi and z;#phi;z;Mean")
g3d.SetMarkerStyle(20)
g3d.SetMarkerColor(root.kBlue)
# Fit the 3D graph
g3d.Fit(fit_func, "Q")  # "Q" for quiet

# Draw the points and the fit surface
g3d.Draw("P0 surf1")
fit_func.Draw("same surf1")

c3d.Draw()

In [17]:
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 4.2 ± 1
Parameter [1]: -2 ± 0.13
Parameter [2]: 0.97 ± 3.2
Parameter [3]: -1.1 ± 0.058
Parameter [4]: -0.1 ± 0.3


In [18]:
# Fill a TH3D with (phi, z, mean) and fit 2D slices with the given function

# Define binning
n_bins_phi = 20
n_bins_z = len(z_means)
phi_min, phi_max = 2, 3.7
z_min, z_max = min(z_means), max(z_means)
mean_min, mean_max = -0.002, 0.002

th3 = root.TH3D("th3", "Mean vs #phi and z;#phi;z;Mean", n_bins_phi, pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], n_bins_z, z_min, z_max, 100, mean_min, mean_max)

# Fill TH3D
for ized, z in enumerate(z_bins):
    n_phi = len(Means[ilayer][ized])
    for iphi in range(n_phi):
        phi = phis[ilayer][ized][iphi]
        mean = Means[ilayer][ized][iphi]
        mean_err = 1./Means_err[ilayer][ized][iphi]
        th3.Fill(phi, z_means[ized], mean, int_fg[ilayer][ized][iphi])

c1 = root.TCanvas("c1", "3D Histogram", 800, 600)
th3.Draw("LEGO2")  # Draw the 3D histogram
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", -2, 4, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these
th3.Fit(fit_func, "Q")  # Fit the 3D histogram
fit_func.Draw("same surf1")
c1.Draw()

# fit_results now contains the fit parameters for each z slice

In [19]:
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 3.8 ± 0.84
Parameter [1]: -1.6 ± 0.17
Parameter [2]: 4.4 ± 4.1
Parameter [3]: -1.1 ± 0.039
Parameter [4]: 0.37 ± 0.4
